# Motivation

This notebooks serves a simple logfile patching purpose. As there are different ways to define the interval when a model
is the `most recent` model for a certain interval, we allow patching the logfile to the desired definition.

By default our pipeline assumes a model is most recent for the time AFTER the training interval.
However, sometimes we want to consider the model most recent for the time DURING the training interval.

This script mutates the `most_recent_model` field in the logfile to the non-default 
definition (during training interval).

In [4]:
from modyn.supervisor.internal.pipeline_executor.models import PipelineLogs


from modyn.supervisor.internal.grpc.enums import PipelineStage
# fill missing times in cumulative plot


from analytics.app.data.transform import logs_dataframe
from pathlib import Path


%load_ext autoreload
%autoreload 2

/Users/robinholzinger/micromamba/envs/modyn/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data loading

In [5]:
# VARIABLES

pipeline_logfile = Path("/Users/robinholzinger/robin/dev/eth/modyn-2/.analytics.log/.data/pipeline_6o/pipeline.log")

In [6]:
logs = PipelineLogs.model_validate_json(pipeline_logfile.read_text())


In [7]:
trains = [(l_ for l_ in logs.supervisor_logs.stage_runs if l_.id == PipelineStage.HANDLE_SINGLE_TRIGGER.name)]
evals = [(l_ for l_ in logs.supervisor_logs.stage_runs if l_.id == PipelineStage.EVALUATE_SINGLE.name and l_.info.eval_request.dataset_id == "cglm_landmark_min25-test")]


In [8]:
from analytics.app.data.transform import dfs_models_and_evals

df_logs = logs_dataframe(logs)
# max_timestamp = df_logs["sample_time"].max()
max_timestamp = df_logs["sample_time"].max()
df_models, eval_requests, evals_metrics = dfs_models_and_evals(logs, max_timestamp)

# Data exploration

In [9]:
df_models

,id,start,end,duration,batch_idx,sample_idx,sample_time,trigger_idx,trigger_id,training_id,id_model,train_start,train_end,real_train_end,num_batches,num_samples,usage_start,usage_end,training_idx,model_idx
0,STORE_TRAINED_MODEL,2024-06-19 22:24:06.894452,2024-06-19 22:24:08.820230,0 days 00:00:01.925790,0,2505174,1041635530,0,0,1,2,2003-01-03 23:12:10,2003-01-03 23:12:10,2004-09-08 08:49:26,0,0,2004-09-08 08:49:27,2005-01-03 00:02:53,1,1
1,STORE_TRAINED_MODEL,2024-06-19 22:24:45.148463,2024-06-19 22:24:46.057907,0 days 00:00:00.909457,0,2505175,1094633367,1,1,5,6,2004-09-08 08:49:27,2005-01-02 19:56:36,2005-01-03 00:02:53,0,0,2005-01-03 00:02:54,2006-01-02 23:47:30,2,2
9,STORE_TRAINED_MODEL,2024-06-19 22:27:01.800712,2024-06-19 22:27:03.312885,0 days 00:00:01.512185,0,2505289,1104710574,2,2,9,10,2005-01-03 00:02:54,2006-01-02 20:09:56,2006-01-02 23:47:30,23,14720,2006-01-02 23:47:31,2007-01-02 23:15:05,3,3
10,STORE_TRAINED_MODEL,2024-06-19 22:31:40.920285,2024-06-19 22:31:42.493102,0 days 00:00:01.572831,0,2508311,1136245651,3,3,13,14,2006-01-02 23:47:31,2007-01-02 23:05:38,2007-01-02 23:15:05,67,42880,2007-01-02 23:15:06,2008-01-03 00:15:39,4,4
11,STORE_TRAINED_MODEL,2024-06-19 22:39:31.417509,2024-06-19 22:39:33.064600,0 days 00:00:01.647102,0,2517000,1167779706,4,4,17,19,2007-01-02 23:15:06,2008-01-02 22:12:29,2008-01-03 00:15:39,125,80000,2008-01-03 00:15:40,2009-01-01 23:28:25,5,5
12,STORE_TRAINED_MODEL,2024-06-19 22:44:49.222251,2024-06-19 22:44:50.756528,0 days 00:00:01.534291,0,2533014,1199319340,5,5,22,21,2008-01-03 00:15:40,2009-01-01 21:20:27,2009-01-01 23:28:25,78,49920,2009-01-01 23:28:26,2010-01-01 23:15:21,6,6
13,STORE_TRAINED_MODEL,2024-06-19 22:51:20.098923,2024-06-19 22:51:22.275917,0 days 00:00:02.177006,0,2553076,1230852506,6,6,24,25,2009-01-01 23:28:26,2010-01-01 23:12:08,2010-01-01 23:15:21,101,64640,2010-01-01 23:15:22,2011-01-01 23:12:57,7,7
14,STORE_TRAINED_MODEL,2024-06-19 22:59:05.815245,2024-06-19 22:59:07.376643,0 days 00:00:01.561410,1,2578993,1287744982,7,7,28,28,2010-01-01 23:15:22,2011-01-01 23:09:18,2011-01-01 23:12:57,127,81280,2011-01-01 23:12:58,2012-01-01 23:16:38,8,8
15,STORE_TRAINED_MODEL,2024-06-19 23:11:57.112704,2024-06-19 23:11:58.705946,0 days 00:00:01.593258,1,2585403,1293923578,8,8,31,32,2011-01-01 23:12:58,2012-01-01 23:10:42,2012-01-01 23:16:38,233,149120,2012-01-01 23:16:39,2012-12-31 23:19:11,9,9
16,STORE_TRAINED_MODEL,2024-06-19 23:23:47.202400,2024-06-19 23:23:48.854301,0 days 00:00:01.651909,2,2645280,1348679446,9,9,35,35,2012-01-01 23:16:39,2012-12-31 22:34:40,2012-12-31 23:19:11,195,124800,2012-12-31 23:19:12,2013-12-31 23:37:22,10,10


In [10]:
eval_requests

,id,start,end,duration,batch_idx,sample_idx,sample_time,trigger_idx,trigger_id,training_id,...,currently_active_model,currently_trained_model,eval_handler,dataset_id,interval_start,interval_end,num_samples,interval_center,training_idx,model_idx
0,EVALUATE_SINGLE,2024-06-20 10:05:14.964348,2024-06-20 10:06:08.413624,0 days 00:00:53.449260,-1,-1,-1,-1,0,1,...,False,True,exactmatrix,cglm_hierarchical_min25-test,2004-01-01,2004-12-31,15,2004-07-01 12:00:00,1,1
0,EVALUATE_SINGLE,2024-06-20 10:08:14.089618,2024-06-20 10:08:55.529398,0 days 00:00:41.439759,-1,-1,-1,-1,16,57,...,False,False,exactmatrix,cglm_hierarchical_min25-test,2004-01-01,2004-12-31,15,2004-07-01 12:00:00,17,17
0,EVALUATE_SINGLE,2024-06-20 10:07:36.494745,2024-06-20 10:08:18.286228,0 days 00:00:41.791456,-1,-1,-1,-1,15,53,...,False,False,exactmatrix,cglm_hierarchical_min25-test,2004-01-01,2004-12-31,15,2004-07-01 12:00:00,16,16
0,EVALUATE_SINGLE,2024-06-20 10:07:36.144441,2024-06-20 10:08:18.152381,0 days 00:00:42.007914,-1,-1,-1,-1,14,50,...,False,False,exactmatrix,cglm_hierarchical_min25-test,2004-01-01,2004-12-31,15,2004-07-01 12:00:00,15,15
0,EVALUATE_SINGLE,2024-06-20 10:07:34.872132,2024-06-20 10:08:16.628708,0 days 00:00:41.756554,-1,-1,-1,-1,13,47,...,False,False,exactmatrix,cglm_hierarchical_min25-test,2004-01-01,2004-12-31,15,2004-07-01 12:00:00,14,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,EVALUATE_SINGLE,2024-06-20 11:18:23.972007,2024-06-20 11:19:05.097838,0 days 00:00:41.125799,-1,-1,-1,-1,1,5,...,False,False,exactmatrix,cglm_hierarchical_min25-test,2019-12-28,2020-12-27,79,2020-06-27 12:00:00,2,2
0,EVALUATE_SINGLE,2024-06-20 11:18:01.961310,2024-06-20 11:18:42.478436,0 days 00:00:40.517102,-1,-1,-1,-1,0,1,...,False,False,exactmatrix,cglm_hierarchical_min25-test,2019-12-28,2020-12-27,79,2020-06-27 12:00:00,1,1
0,EVALUATE_SINGLE,2024-06-20 11:20:35.560337,2024-06-20 11:21:16.979049,0 days 00:00:41.418689,-1,-1,-1,-1,15,53,...,True,False,exactmatrix,cglm_hierarchical_min25-test,2019-12-28,2020-12-27,79,2020-06-27 12:00:00,16,16
0,EVALUATE_SINGLE,2024-06-20 11:19:13.107004,2024-06-20 11:19:54.693075,0 days 00:00:41.586053,-1,-1,-1,-1,7,28,...,False,False,exactmatrix,cglm_hierarchical_min25-test,2019-12-28,2020-12-27,79,2020-06-27 12:00:00,8,8


In [11]:
eval_requests[eval_requests["currently_active_model"]]

,id,start,end,duration,batch_idx,sample_idx,sample_time,trigger_idx,trigger_id,training_id,...,currently_active_model,currently_trained_model,eval_handler,dataset_id,interval_start,interval_end,num_samples,interval_center,training_idx,model_idx
0,EVALUATE_SINGLE,2024-06-20 10:08:16.628770,2024-06-20 10:09:00.018658,0 days 00:00:43.389864,-1,-1,-1,-1,0,1,...,True,False,exactmatrix,cglm_hierarchical_min25-test,2004-12-31,2005-12-31,324,2005-07-01 12:00:00,1,1
0,EVALUATE_SINGLE,2024-06-20 10:11:13.023427,2024-06-20 10:11:58.237048,0 days 00:00:45.213597,-1,-1,-1,-1,1,5,...,True,False,exactmatrix,cglm_hierarchical_min25-test,2005-12-31,2006-12-31,947,2006-07-01 12:00:00,2,2
0,EVALUATE_SINGLE,2024-06-20 10:14:49.269512,2024-06-20 10:15:39.747761,0 days 00:00:50.478225,-1,-1,-1,-1,2,9,...,True,False,exactmatrix,cglm_hierarchical_min25-test,2006-12-31,2007-12-31,1788,2007-07-01 12:00:00,3,3
0,EVALUATE_SINGLE,2024-06-20 10:18:22.883266,2024-06-20 10:19:14.151931,0 days 00:00:51.268645,-1,-1,-1,-1,3,13,...,True,False,exactmatrix,cglm_hierarchical_min25-test,2007-12-31,2008-12-30,2184,2008-06-30 12:00:00,4,4
0,EVALUATE_SINGLE,2024-06-20 10:22:32.688110,2024-06-20 10:23:28.176396,0 days 00:00:55.488266,-1,-1,-1,-1,4,17,...,True,False,exactmatrix,cglm_hierarchical_min25-test,2008-12-30,2009-12-30,2872,2009-06-30 12:00:00,5,5
0,EVALUATE_SINGLE,2024-06-20 10:26:28.418194,2024-06-20 10:27:26.916849,0 days 00:00:58.498628,-1,-1,-1,-1,5,22,...,True,False,exactmatrix,cglm_hierarchical_min25-test,2009-12-30,2010-12-30,3518,2010-06-30 12:00:00,6,6
0,EVALUATE_SINGLE,2024-06-20 10:31:21.800921,2024-06-20 10:32:35.482845,0 days 00:01:13.681901,-1,-1,-1,-1,6,24,...,True,False,exactmatrix,cglm_hierarchical_min25-test,2010-12-30,2011-12-30,6725,2011-06-30 12:00:00,7,7
0,EVALUATE_SINGLE,2024-06-20 10:36:43.485577,2024-06-20 10:37:53.956958,0 days 00:01:10.471354,-1,-1,-1,-1,7,28,...,True,False,exactmatrix,cglm_hierarchical_min25-test,2011-12-30,2012-12-29,5474,2012-06-29 12:00:00,8,8
0,EVALUATE_SINGLE,2024-06-20 10:42:21.001936,2024-06-20 10:43:41.964042,0 days 00:01:20.962089,-1,-1,-1,-1,8,31,...,True,False,exactmatrix,cglm_hierarchical_min25-test,2012-12-29,2013-12-29,7365,2013-06-29 12:00:00,9,9
0,EVALUATE_SINGLE,2024-06-20 10:48:17.857491,2024-06-20 10:49:33.318476,0 days 00:01:15.460964,-1,-1,-1,-1,9,35,...,True,False,exactmatrix,cglm_hierarchical_min25-test,2013-12-29,2014-12-29,6508,2014-06-29 12:00:00,10,10


# Patch logfile

In [ ]:
for eval_log in logs.supervisor_logs.stage_runs:
    if eval_log.id == PipelineStage.EVALUATE_SINGLE.name:
        # Let's throw away all information about the most recent model, let's rebuild it
        eval_log.info.eval_request.currently_active_model = False

        # For a fixed interval the evaluation request of a certain model is the most recent, if the model training
        # interval center lies within the evaluation interval.
        # Note: this is not a generic solution, but works for the slicing case with fixed evaluation and trigger
        # intervals in the same order of magnitude.
        model_row = df_models[df_models["id_model"] == eval_log.info.eval_request.id_model]
        assert len(model_row) == 1

        training_center = (model_row.iloc[0]["train_start"].to_pydatetime().timestamp() + model_row.iloc[0]["train_end"].to_pydatetime().timestamp()) / 2
        eval_log.info.eval_request.currently_active_model = eval_log.info.eval_request.interval_start <= training_center <= eval_log.info.eval_request.interval_end
        eval_log.info.eval_request.currently_trained_model = eval_log.info.eval_request.interval_start <= training_center <= eval_log.info.eval_request.interval_end

In [ ]:
# Write results back
pipeline_logfile.write_text(logs.model_dump_json(by_alias=True))